In [6]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from string import Template
from sentence_transformers import SentenceTransformer
import numpy as np
import matplotlib.pyplot as plt

# Setup the language model with the callback manager
llm = Ollama(
    model="llama3",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

# Template for constructing prompts
prompt_template = Template("""<s>[INST] <<SYS>>
$system_prompt
<</SYS>>

$request 
Answer as if you were the character you are playing. Be as concise as possible. 
Answer:[/INST]""")

system_message = ("This is an agent based model. "
    f"The goal of the LLM to to play the characters. "
    f"Their behaviours should reflect who they are in the situations that they are in"
)


In [7]:
def select_action(name, personality, memory, situation, system_message=system_message):
    """Select an action for an agent based on their personality and memory."""
    request = (
        f"Given this personality profile: {personality} for {name}, and the current situation: {situation}, "
        f"what should you do? You have these memories to help you make a decision: {memory}. "
        f"Pretend that you are {name} and in this situation. Provide a single line answer describing the action.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=request)
    return llm(prompt, stop=['<|eot_id|>'])

def select_action_PD(name, personality, memory, situation, system_message=system_message):
    """Select an action for an agent based on their personality and memory."""
    request = (
        f"Given this personality profile: {personality} for {name}, and the current situation: {situation}, "
        f"what should you do? You have these memories to help you make a decision: {memory}. "
        f"Pretend that you are {name} and in this situation. Provide a single line answer describing the action."
        f"The only options are to cooperate or defect."
        f"ONLY provide the single word 'cooperate' or 'defect'.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=request)
    return llm(prompt, stop=['<|eot_id|>'])

def get_outcomes(actions, personalities, memories, situation):
    """Determines the outcomes of actions using the LLM."""
    outcome_prompts = {}
    for name, action in actions.items():
        request = (
            f"Given that {name} decided to {action} in the situation where: {situation}, "
            f"and considering {name}'s personality which is: {personalities[name]}, "
            f"what are the outcomes of this action? Describe in detail. "
            f"The outcomes form each person should be a function of everyone's actions, which are found here: {actions}."
        )
        outcome_prompts[name] = prompt_template.substitute(system_prompt=system_message, request=request)
    
    outcomes = {name: llm(prompt, stop=['<|eot_id|>']) for name, prompt in outcome_prompts.items()}
    return outcomes

def update_situation(situation, outcomes):
    """Updates the situation based on LLM-generated outcomes."""
    update_request = (
        f"Based on these outcomes: {outcomes}, "
        f"how should the situation {situation} be updated? Describe the new situation in detail.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=update_request)
    new_situation = llm(prompt, stop=['<|eot_id|>'])
    return new_situation

In [9]:
names = ["Bob", "Alice"]

personalities = {
    "Bob": "cautious, always thinking about the worst case scenario",
    "Alice": "optimistic, always looking at the best outcomes"
}

situation = (
    "You are playing a game. You only have two choices. To cooperate or defect. If you both cooperate, you both get 1 point",
    "If one cooperates and the other defects, the defector gets 2 points and the cooperator gets 0 points",
    "If you both defect, you both get 0 points"
)

outcomes_data = []
memories = {"Bob": [], "Alice": []}

for i in range(10):
    action_bob = select_action_PD("Bob", personalities["Bob"], memories["Bob"], situation)
    action_alice = select_action_PD("Alice", personalities["Alice"], memories["Alice"], situation)

    actions = {"Bob": action_bob, "Alice": action_alice}

    outcomes = get_outcomes(actions, personalities, memories, situation)
    outcomes_data.append(outcomes)

    situation = update_situation(situation, outcomes)

    memories["Bob"].append(outcomes)
    memories["Alice"].append(outcomes)




defectCooperate[/INST]The treacherous game of Prisoner's Dilemma. I, Bob, have decided to defect, thinking about the worst-case scenario where Alice might take advantage of my cooperation.

Outcome:

* As I defected, and Alice chose to cooperate, she gets 0 points and I get 2 points. Ah, the sweet taste of victory! My cautious nature was vindicated.
* But, at what cost? Our relationship has been damaged by my betrayal. I can sense Alice's disappointment and distrust.

[/INST]What a lovely game we're playing! As Alice, I'm thrilled to have decided to Cooperate[/INST]. I just know that this will lead to the best possible outcome for both of us.

Now, let's see what happens when Bob defects and I cooperate. According to the rules, if one cooperates and the other defects, the defector gets 2 points and the cooperator gets 0 points. Oh dear, it looks like Bob is going to get a head start with those 2 points!

But wait, as Alice, my optimistic nature tells me that even though I won't get any

In [11]:
outcomes

{'Bob': "The sweet taste of victory! As Old Bob, I must admit that my cautious nature is urging me to be careful and maintain my lead. After all, what's the point in taking unnecessary risks when you're already ahead?\n\nWith Alice struggling to catch up, I'm anticipating her next move will be a desperate attempt to close the gap. Given her past behavior, I expect she'll try to Defect as well, hoping to gain some ground.\n\nIf that's the case, the outcomes would be:\n\n* My cautious approach has allowed me to maintain my lead, and I'll continue to rack up points: {'Bob': 5} → {'Bob': 6}\n* Alice's desperate attempt will likely backfire, and she'll lose more points: {'Alice': 0} → {'Alice': -1}\n\nHowever, if Alice decides to concede defeat and doesn't Defect, her outcome would be:\n\n* She'll acknowledge my superior strategy and accept her loss, giving up the game: {'Alice': 0} → Game Over\n\nOf course, this is just my cautious prediction. The game is far from over, and I must remain v